# Prédiction de la valeur foncière d'un bien immobilier à Paris

Dans ce projet, nous avons pour but de prédire la valeur foncière d'un bien immobilier à Paris. Pour ce faire, nous allons utiliser des données disponibles en opendata sur deux sites et nous allons faire du scraping pour obtenir une autre base de données.

### Récuperation de la base de données DVF

Tout d'abord, nous avons récupéré la base de données que nous nommerons "DVF" sur le site disponible [ici](https://cerema.app.box.com/v/dvfplus-opendata). Ce site contient de nombreuses bases de données contenant des informations sur des mutations de biens immobiliers (tel que la date de la vente, le type du bien, sa localisation ect...). Les bases de données que nous avons récupérées sont uniquement celles concernant les mutations à Paris. Nous avons ensuite dû concaténer toutes les bases de données récupérées.
Voici le [notebook](../notebook/extraction_merging.ipynb) dans lequel ces étapes ont été réalisées.

### Analyse de la base de données DVF

Ensuite, nous avons analysé cette base de donnée. Avec une première analyse sur le type du bien et la nature de la mutation, nous avons décidé de ne garder que les observations concernant les **ventes** de **maisons** ou d'**appartements**. 

Nous avons alors étudier les variables. Nous nous sommes rendus compte que certaines variables prenaient très souvent la même valeur. Nous avons décidé de supprimer ces variables. Puis, nous avons étudié les valeurs manquantes de la variable concernant la valeur foncière. Leur fréquence étant très faible, nous avons décidé de les enlever du dataset. Après analyse des variables restantes, nous nous sommes rendu compte que la moyenne des valeurs foncières par année est en croissance de 2016 à 2019. Cette évolution temporelle est un problème car notre objectif étant la prédiction, nos modèles risquent d'avoir des problèmes s'il doit prédire la valeur d'un bien en 2020 ou plus tard. Il faudra donc trouver une solution pour enlever cette tendance. Enfin nous avons étudié les corrélations entre nos variables.

Nous avons ensuite pris en main les différents datasets geopandas en les concaténant et en faisant une rapide analyse.

Toutes ces étapes se trouve [ici](../notebook/analysis_and_treat.ipynb)

### Webscraping de ville-idéale

[ville-idéale](https://www.ville-ideale.fr/) est un site internet sur lequel des internautes peuvent noter leur quartier sur plusieurs critères (santé, culture, sécurité...). Nous avons trouvé intéressant d'utiliser ses données pour de la visualisation. C'est pourquoi nous avons fait du webscraping pour récupérer les données pour les quartiers de Paris.

Ce webscraping est effectué dans ce [script](../script/ville_ideale_scraping.py)

### Récupération et analyse de la base de données Filosofi

Nous nous sommes alors penchés sur une autre base de données : Filosofi. Cette base de données est en opendata sur le [site](https://www.insee.fr/fr/statistiques/4176290?sommaire=4176305) de l'INSEE. Elle est notamment constituée d'indicateur de niveau de vie, d'inégalité et de pauvreté. Nous avons uniquement récupéré les données pour Paris. Il s'agit d'un échelle carroyée de 200 mètres.

Nous avons analysé cette table dans sa globalité puis par carreaux.

Le notebook correspondant est [ici](../notebook/analyse_filosofi.ipynb).

### Lien entre la position d'un bien dans la base DVF et le carreau auquel il appartient dans la base Filosofi

Il a ensuite fallu associer à chaque mutation de la base DVF un carreau associé de la base Filosofi. Ceci permet de faire la jointure entres les deux tables.

Voici le [notebook](../notebook/jointure_mutation_filosofi.ipynb) associé.

### Récupération et utilisation de la base de données BPE

La base BPE (base permanente des équipements) contient des informations sur le niveau d'équipements et de services rendus à la population sur un territoire. Il est disponible sur le [site](https://www.insee.fr/fr/statistiques/3568656) de l'INSEE. Il y a plus de 130 types d'équipements. Nous nous sommes évidemment restreints aux données parisiennes. Nous avons utilisé ces données pour calculer la distance de l'équipement le plus proche, pour chaque mutation et pour chaque type d'équipements. Nous avons retirés des équipements (qui semblaient peu influant dans le prix d'un bien immobilier) et fait des regroupements d'équipements.

Ces étapes sont disponibles dans ce [notebook](../notebook/localisation_service_bpe.ipynb)

### Analyse et mise en forme avant la modélisation

Nous avons maintenant une base de données fusionnant des informations des quatres bases de données citées précédemment.

Nous avons alors essayé de corriger l'inflation. Pour ce faire, nous avons fait deux hypothèses :  
- la variation de la moyenne annuelle du prix des biens immobiliers à Paris est uniquement due à l'inflation.
- la moyenne annuelle de la valeur foncière est linéaire par rapport à l'année de la mutation entre 2014 et 2016 ainsi qu'entre 2016 et 2019

Nous avons alors pu corriger cette inflation.

Nous nous sommes ensuite penchés sur les valeurs manquantes provenant des deux dernières bases de données (Filosofi et BPE). Là encore, les observations avec des données manquantes étant très rares, nous avons décidé de les supprimer.

Puis, nous avons étudié les valeurs extrêmes de nos variables. Nous avons notamment remarqué que les maisons représentaient une grande part des valeurs extrêmes supérieures de la valeur foncière. De plus, c'est une très faible proportion de nos données. Nous avons donc à cette étape décidé de ne garder que les **appartements**.

Enfin, nous avons fait le découpage de notre base de données en trois : sous-ensemble d'entraînement (60%), sous-ensemble de test (20%) et sous-ensemble de validation (20%).

Le notebook correspondant est [ici](../notebook/analyse_mise_en_forme_avant_modelisation.ipynb).

### Modélisation

#### SVR (support vector regression)

#### Forêt aléatoire

#### XGBoost